In [37]:
import pandas as pd
from typing import List

from analysis.utils.cleaning import lower_case_and_strip_spaces

### The Data and what we want to do with it:

These datasets where originally from Kaggle here is the description they gave:
This dataset contains 27753444 ratings across 58098 movies. These data sets were created by 283228 users between January 09, 1995 and September 26, 2018. This dataset was generated on September 26, 2018.

### Reading the Data and Initial Stats

In [4]:
movies_df: pd.DataFrame = pd.read_csv('input/all_movies.csv')
movies_df.sample(20)

,movieId,title,genres
36737,145222,Bedlam (2015),Thriller
27708,123254,State Fair (1962),(no genres listed)
55004,186405,Ben 10 - Alien Swarm (2009),Adventure
27385,122533,Breach of Trust (1995),Action|drama|THRILLER
23402,109935,The Artist and the Model (2012),Drama
2295,2379,Police Academy 2: Their First Assignment (1985),Comedy|crime
50359,176165,Cop and a Half: New Recruit (2017),Action|children|COMEDY|crime
18160,90405,In Time (2011),Crime|sci-fi|THRILLER
39854,152894,Lakeer - Forbidden Lines (2004),Action|drama|ROMANCE
49275,173771,McLaren (2016),Action|documentary|DRAMA


In [6]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|animation|CHILDREN|comedy|Fantasy
1,2,Jumanji (1995),Adventure|children|FANTASY
2,3,Grumpier Old Men (1995),Comedy|romance
3,4,Waiting to Exhale (1995),Comedy|drama|ROMANCE
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
58093,193876,The Great Glinka (1946),(no genres listed)
58094,193878,Les tribulations d'une caissière (2011),Comedy
58095,193880,Her Name Was Mumu (2016),Drama
58096,193882,Flora (2017),Adventure|drama|HORROR|sci-fi


In [7]:
ratings_df = pd.read_csv('input/ratings.csv')
ratings_df.sample(20)

,userId,movieId,rating,timestamp
8102165,83406,1234,5.0,1095728493
12400877,127058,367,2.0,1111287555
17038902,174150,608,5.0,945700940
2669324,27508,357,3.0,846323768
23571102,240828,40826,2.5,1136767320
27431524,279863,3608,3.0,1366525194
20661989,211008,529,5.0,845075838
11705337,120061,65,3.0,1152720027
17019370,173990,63859,3.5,1272740060
18222198,185980,43897,3.5,1299948170


## Step 1: Cleaning
You shouldn't necessarily assume that your data is good.  It could be very sparse and not have much there. There could be duplication, poorly recorded or empty values, or with large text there could be a lot of garbage in there if it was an open text field

Some things we'll do here:
1. Get the number of rows and columns by looking at the shape
2. Determine the number of non-null rows for each given column (This would tell us if a column is especially sparse)
3. Check for duplicate rows

As we find things that need to be cleaned (bad text, duplicates etc.) we will write tested cleaning functions to cleanup our input data.

In [8]:
movies_df.shape

(58098, 3)

In [9]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


#### Checking for Duplicates

First we need to define what a "duplicate" is.

Let's do some cleanup.

Now let's determine what is happening to cause the shape discrepency

Here we have a situation where 56 things are duplicated based on their title but the two rows have different genres.  How can we easily find an example movie where this is the case?

Let's check out an example:

### Ratings Data Frame
Same Steps
1. Easily get the number of rows and columns by looking at the shape
2. Determine the number of non-null rows for each given column
3. Check for duplicate rows

Here we should again define what it means to be a "duplicate":

## Step 2: Feature Preparation
We want to be able to recommend movies to users. What are some things we could do next?

Answer:
- Join movies to ratings
- Group by movie title to find an average rating for each movie

Here we will left join movies to their ratings.  If we expect all movies to have ratings than we can start with an assertion that indicates that.

Let's find the movies that were missing ratings (if any).

Now let's keep only movies that did have ratings

Next let's get an average rating for each movie by grouping by title and creating a list of the ratings and taking the first instance of a genres list.

Let's think about our recommendation engine now again.  Let's say that we want to recommend movies to by recommending the movies with the most similar genres list.

In order to to use TF IDF we need a list of all the "words" (genres) used in our corpus.  This is easy for us to do.  We can make a list of all the genres by:
1. Creating a column with a list of genres
2. Grouping by the genre
3. Aggregating the results
4. Transforming the resulting series into a list of genres

Column with list of genres

Group by Genre

Aggregrate the results

create a list of genres

Let's turn our list of genres into a space separated list of genres (as if they were words in a sentence)

reset our index again

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 1),min_df=0, stop_words='english', vocabulary=all_genres_list)
tfidf_matrix = tf.fit_transform(movies_with_ratings_combined_df['genres_as_string'])

To see what our matrix looks like we can do the following:

In [89]:
pd.DataFrame(tfidf_matrix.toarray())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,0.0,0.000000,0.000000,0.775664,0.000000,0.0,0.631147,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.292236,0.0,0.0,0.529943,0.0,0.0,0.645142,0.000000,0.0,0.466423,0.000000,0.0
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50091,0.0,0.0,0.000000,0.000000,0.440962,0.671268,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.595778,0.0,0.000000,0.000000,0.0
50092,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.378841,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.925462,0.0
50093,0.0,0.0,0.000000,0.000000,0.486407,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.873732,0.000000,0.0,0.000000,0.000000,0.0
50094,0.0,0.0,0.707986,0.706226,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


Now let's calculate the dot product of the tfidf_martix with itself in order to get a cosine similarity matrix.

In [90]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

 What do we expect the dimensions of this matrix to be?

In [91]:
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.37728826, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.37728826, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        1.        ]])

Notice that this matrix also has 1's along the diagonals. Why is that?

In [92]:
cosine_sim.shape

(50096, 50096)

The next part you can Test Drive because now we'll add logic to grab the top 20 movie titles by index

We can build this in a separate module and import it here to see some results

In [93]:
from analysis.utils.recommendation import get_similar_movies

similar_movies = get_similar_movies('Toy Story (1995)', cosine_sim, movies_with_ratings_combined_df)
similar_movies

['Antz (1998)', 'Asterix and the Vikings (Astérix et les Vikings) (2006)']

THAT'S IT! :)